## RL Hacking 3

Hack in this direction as long as it makes sense: 

- [ ]  Setup a basic LLM (maybe QWEN 0.6B)
- [ ]  A math dataset
- [ ]  Run various policy gradient methods to teach the model to reason (GRPO, Raschka’s simpler GRPO, Dr. GRPO, PPO, Vanilla Policy gradient, maybe DPO, maybe RPOO)
- [ ]  How do implementations perform and vary? Do components make sense? Can we replicated some version of the “aha moment”? (although that’s apparently not an emergent RL property?) Does the way the math is setup make sense? does Karpathy’s simplified explanation make it easier? Would demoing on or more of these algorithms on in a game setting make more sense?

I'd prefer to use standard HF QWEN here, but let's roll with Raschka's rooling for a bit
- https://github.com/rasbt/reasoning-from-scratch/tree/main/ch06
- https://github.com/McGill-NLP/nano-aha-moment/blob/main/nano_r1.ipynb
- https://github.com/rasbt/reasoning-from-scratch/blob/main/ch07/01_main-chapter-code/ch07_main.ipynb

In [1]:
import torch

from reasoning_from_scratch.ch02 import get_device
from reasoning_from_scratch.ch03 import (
     load_model_and_tokenizer
)

device = get_device()
# device = torch.device("cpu")

model, tokenizer = load_model_and_tokenizer(
    which_model="base",
    device=device,
    use_compile=False
)

Using NVIDIA CUDA GPU
✓ qwen3/qwen3-0.6B-base.pth already up-to-date


In [3]:
from reasoning_from_scratch.ch03 import render_prompt
from reasoning_from_scratch.ch04 import (
    generate_text_stream_concat_flex,
    generate_text_top_p_stream_cache
)

raw_prompt = (
    "Half the value of $3x-9$ is $x+37$. "
    "What is the value of $x$?"
)
prompt = render_prompt(raw_prompt)

torch.manual_seed(0)
response = generate_text_stream_concat_flex(
    model, tokenizer, prompt, device,
    max_new_tokens=2048, verbose=True,
    generate_func=generate_text_top_p_stream_cache,
    temperature=0.9,
    top_p=0.9
)

 46

In [4]:
response

' 46'

In [5]:
import json
import requests
from pathlib import Path

def load_math_train(local_path="math_train.json", save_copy=True):
    local_path = Path(local_path)
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "math_full_minus_math500/refs/heads/main/"
        "math_full_minus_math500.json"
    )

    if local_path.exists():
        with local_path.open("r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        data = r.json()

        if save_copy:  # Saves a local copy
            with local_path.open("w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)

    return data

In [6]:
math_train = load_math_train()

print("Dataset size:", len(math_train))

Dataset size: 12000


In [7]:
from pprint import pprint

pprint(math_train[4])

{'answer': '6',
 'level': 'Level 3',
 'problem': 'Sam is hired for a 20-day period. On days that he works, he earns '
            '$\\$$60. For each day that he does not work, $\\$$30 is '
            'subtracted from his earnings. At the end of the 20-day period, he '
            'received $\\$$660. How many days did he not work?',
 'solution': 'Call $x$ the number of days Sam works and $y$ the number of days '
             'he does not. We can set up the following system of equations to '
             'represent the given information: \\begin{align*}\n'
             'x+y &= 20 \\\\\n'
             '60x - 30y &= 660 \\\\\n'
             '\\end{align*} The first equation represents the total number of '
             'days Sam works, and the second equation represents his total '
             'profit. Solving for $x$ in the first equation yields $x = 20 - '
             'y$. Substituting into the second equation gives $60(20-y) - 30y '
             '= 660$. Canceling a factor of $10$ an

In [8]:
from reasoning_from_scratch.qwen3 import KVCache
from reasoning_from_scratch.ch04 import top_p_filter


@torch.no_grad()
def sample_response(
    model,
    tokenizer,
    prompt,
    device,
    max_new_tokens=512,
    temperature=0.8,
    top_p=0.9,
):
    input_ids = torch.tensor(
        tokenizer.encode(prompt),
        device=device
        )

    cache = KVCache(n_layers=model.cfg["n_layers"])
    model.reset_kv_cache()
    logits = model(input_ids.unsqueeze(0), cache=cache)[:, -1]

    generated = []
    for _ in range(max_new_tokens):
        if temperature and temperature != 1.0:
            logits = logits / temperature

        probas = torch.softmax(logits, dim=-1)
        probas = top_p_filter(probas, top_p)
        next_token = torch.multinomial(
            probas.cpu(), num_samples=1
        ).to(device)

        if (
            tokenizer.eos_token_id is not None
            and next_token.item() == tokenizer.eos_token_id
        ):
            break
        generated.append(next_token.item())
        logits = model(next_token, cache=cache)[:, -1]

    full_token_ids = torch.cat(
        [input_ids,
         torch.tensor(generated, device=device, dtype=input_ids.dtype),]
    )
    return full_token_ids, input_ids.numel(), tokenizer.decode(generated)

In [9]:
torch.manual_seed(0)

raw_prompt = (
    "Half the value of $3x-9$ is $x+37$. "
    "What is the value of $x$?"
)
prompt = render_prompt(raw_prompt)

token_ids, prompt_len, answer_text = sample_response(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            device=device,
            max_new_tokens=512,
            temperature=0.9,
            top_p=0.9,
        )

print(answer_text)

 46


In [10]:
torch.manual_seed(1)

token_ids, prompt_len, answer_text = sample_response(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            device=device,
            max_new_tokens=512,
            temperature=0.9,
            top_p=0.9,
        )

print(answer_text)

 Let's solve the problem step by step.

1. **Translate the problem into an equation:**
   
   The problem states that half the value of \( 3x - 9 \) is \( x + 37 \). We can translate this into the following equation:
   \[
   \frac{1}{2}(3x - 9) = x + 37
   \]

2. **Simplify the equation:**
   
   Distribute the \( \frac{1}{2} \) on the left side:
   \[
   \frac{3x}{2} - \frac{9}{2} = x + 37
   \]

3. **Eliminate the fraction by multiplying every term by 2:**
   \[
   3x - 9 = 2x + 74
   \]

4. **Isolate the variable \( x \):**
   
   Subtract \( 2x \) from both sides:
   \[
   3x - 2x - 9 = 74
   \]
   \[
   x - 9 = 74
   \]

5. **Solve for \( x \):**
   
   Add 9 to both sides:
   \[
   x = 74 + 9
   \]
   \[
   x = 83
   \]

6. **Write the final answer:**
   
   \[
   \boxed{83}
   \]


In [11]:
torch.manual_seed(5)

token_ids, prompt_len, answer_text = sample_response(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            device=device,
            max_new_tokens=512,
            temperature=0.9,
            top_p=0.9,
        )

print(answer_text)

 Let's solve the problem step by step.

**Given:**
\[
\frac{1}{2} \times (3x - 9) = x + 37
\]

**Step 1: Eliminate the fraction by multiplying both sides by 2.**
\[
2 \times \left( \frac{1}{2} \times (3x - 9) \right) = 2 \times (x + 37)
\]
\[
3x - 9 = 2x + 74
\]

**Step 2: Subtract \(2x\) from both sides to get the \(x\)-terms on one side.**
\[
3x - 2x - 9 = 74
\]
\[
x - 9 = 74
\]

**Step 3: Add 9 to both sides to solve for \(x\).**
\[
x = 74 + 9
\]
\[
x = 83
\]

**Final Answer:**
\[
\boxed{83}
\]


- It is cool that sometimes QWEN kidna reasons here and sometimes it doesn't.
- Wonder how this various across models
- Ok so from here Raschka uses some simulated rollouts to walk through GPRO math, that's cool.
- The general premise of how do we train our model to get the right answer more often is nice
  

In [12]:
from reasoning_from_scratch.ch03 import (
    extract_final_candidate, grade_answer
)

def reward_rlvr(answer_text, ground_truth):
    extracted = extract_final_candidate(
        answer_text, fallback=None  # Require \boxed{}
    )
    if not extracted:
        return 0.0
    correct = grade_answer(extracted, ground_truth)
    return float(correct)

In [13]:
@torch.inference_mode()
def avg_logprob_answer(model, tokenizer, prompt, answer, device="cpu"):

    # Encode prompt and answer tokens separately to get the prompt length later
    prompt_ids = tokenizer.encode(prompt)
    answer_ids = tokenizer.encode(answer)
    full_ids = torch.tensor(prompt_ids + answer_ids, device=device)

    # Same as in calc_next_token_logprobas before
    logits = model(full_ids.unsqueeze(0)).squeeze(0)
    logprobs = torch.log_softmax(logits, dim=-1)

    # Index range for positions corresponding to answer tokens
    start = len(prompt_ids) - 1
    end = full_ids.shape[0] - 1

    # Same as before, except for using start and end
    t_idx = torch.arange(start, end, device=device)
    next_tokens = full_ids[start + 1 : end + 1]
    next_token_logps = logprobs[t_idx, next_tokens]

    # Average over the answer token scores
    return torch.mean(next_token_logps).item()

In [15]:
avg_logprob_val = avg_logprob_answer(
                   model, tokenizer, 
                   prompt=prompt,
                   answer=answer_text,
                   device=device) 

sequence_logprob_val = avg_logprob_val * (len(tokenizer.encode(answer_text)))
print(sequence_logprob_val)

-12.802001953125


In [19]:
#SW - maybe work wiht this more robust version from Raschka? Renaming here

def sequence_logprob(model, token_ids, prompt_len):
    logits = model(token_ids.unsqueeze(0)).squeeze(0).float()
    logprobs = torch.log_softmax(logits, dim=-1)

    # Positions whose next-token probabilities we want
    # These correspond to predicting token_ids[t + 1] from position t
    start = prompt_len - 1
    end = token_ids.shape[0] - 1

    t_idx = torch.arange(start, end, device=token_ids.device)
    next_tokens = token_ids[start + 1 : end + 1]
    next_token_logps = logprobs[t_idx, next_tokens]

    # Sum log-probabilities over the answer tokens
    return torch.sum(next_token_logps)

print(sequence_logprob(model, token_ids, prompt_len))

tensor(-12.7940, device='cuda:0', grad_fn=<SumBackward0>)


In [20]:
def compute_grpo_loss(
    model,
    tokenizer,
    example,
    device,
    num_rollouts=2,
    max_new_tokens=256,
    temperature=0.8,
    top_p=0.9,
):
    assert num_rollouts >= 2
    roll_logps, roll_rewards, samples = [], [], []
    prompt = render_prompt(example["problem"])

    was_training = model.training
    model.eval()

    for _ in range(num_rollouts):
        # Stage 1: generate rollouts
        token_ids, prompt_len, text = sample_response(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            device=device,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
        )
        # Stage 2: compute rewards
        reward = reward_rlvr(text, example["answer"])
        
        # Stage 4: compute logprobs
        logp = sequence_logprob(model, token_ids, prompt_len)

        roll_logps.append(logp)
        roll_rewards.append(reward)
        samples.append(
            {
                "text": text,
                "reward": reward,
                "gen_len": token_ids.numel() - prompt_len,
            }
        )

    if was_training:
        model.train()

    # Stage 2: collect all rewards
    rewards = torch.tensor(roll_rewards, device=device)

    # Stage 3: compute advantages
    advantages = (rewards - rewards.mean()) / (rewards.std() + 1e-4)

    # Stage 4: collect all logprobs
    logps = torch.stack(roll_logps)

    # Stage 5: compute policy gradient loss
    pg_loss = -(advantages.detach() * logps).mean()
    loss = pg_loss  # In the next chapter we add a KL term here

    return {
        "loss": loss.item(),
        "pg_loss": pg_loss.item(),
        "rewards": roll_rewards,
        "advantages": advantages.detach().cpu().tolist(),
        "samples": samples,
        "loss_tensor": loss,
    }

In [21]:
torch.manual_seed(123)

stats = compute_grpo_loss(
    model=model,
    tokenizer=tokenizer,
    example=math_train[4],
    device=device,
    num_rollouts=2,
    max_new_tokens=256,
    temperature=0.8,
    top_p=0.9
)

pprint(stats)

{'advantages': [0.0, 0.0],
 'loss': -0.0,
 'loss_tensor': tensor(-0., device='cuda:0', grad_fn=<NegBackward0>),
 'pg_loss': -0.0,
 'rewards': [0.0, 0.0],
 'samples': [{'gen_len': 3, 'reward': 0.0, 'text': ' 14'},
             {'gen_len': 3, 'reward': 0.0, 'text': ' 4 days'}]}


In [22]:
import time

def train_rlvr_grpo(
    model,
    tokenizer,
    math_data,
    device,
    steps=None,
    num_rollouts=2,
    max_new_tokens=256,
    temperature=0.8,
    top_p=0.9,
    lr=1e-5,
    checkpoint_every=50,
    checkpoint_dir=".",
    csv_log_path=None,

):
    if steps is None:
        steps = len(math_data)

    # Stage 1: initialize optimize
    # (the model was already initialized outside the function)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    model.train()
    current_step = 0
    if csv_log_path is None:
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        csv_log_path = f"train_rlvr_grpo_metrics_{timestamp}.csv"
    csv_log_path = Path(csv_log_path)

    try:
        # Stage 2: Iterate over training steps
        for step in range(steps):

            # Stage 3: Reset loss gradient
            # (it's best practice to do this at the beginning of each step)
            optimizer.zero_grad()

            current_step = step + 1
            example = math_data[step % len(math_data)]

            # Stage 4: calculate GRPO loss
            stats = compute_grpo_loss(
                model=model,
                tokenizer=tokenizer,
                example=example,
                device=device,
                num_rollouts=num_rollouts,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
            )

            # Stage 5: Backward pass to calculate loss gradients
            stats["loss_tensor"].backward()

            # Clip large gradients to improve training stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Stage 6: Update model weights using loss gradients
            optimizer.step()

            # Stage 7: Collect rewards, response lengths, and losses
            reward_avg = torch.tensor(stats["rewards"]).mean().item()
            step_tokens = sum(
                sample["gen_len"] for sample in stats["samples"]
            )
            avg_response_len = (
                step_tokens / len(stats["samples"]) 
                if stats["samples"] else 0.0
            )
            append_csv_metrics(
                csv_log_path, current_step, steps, stats["loss"],
                reward_avg, avg_response_len,
            )

            # Print step metrics
            print(
                f"[Step {current_step}/{steps}] "
                f"loss={stats['loss']:.4f} "
                f"reward_avg={reward_avg:.3f} "
                f"avg_resp_len={avg_response_len:.1f}"
            )

            # Sample outputs (every 10 steps) to check if model
            # generates coherent text
            if current_step % 10 == 0:
                print(f"[Step {current_step}] sample outputs")
                for i, sample in enumerate(stats["samples"][:3]):
                    text = sample["text"].replace("\n", "\\n")
                    print(
                        f"  {i+1}) reward={sample['reward']:.3f} "
                        f"len={sample['gen_len']}: {text}"
                    )
                print()

            # Stage 8: Save model checkpoint
            if checkpoint_every and current_step % checkpoint_every == 0:
                ckpt_path = save_checkpoint(
                    model=model,
                    checkpoint_dir=checkpoint_dir,
                    step=current_step,
                )
                print(f"Saved checkpoint to {ckpt_path}")

    # Save a model checkpoint if we interrupt the training early
    except KeyboardInterrupt:
        ckpt_path = save_checkpoint(
            model=model,
            checkpoint_dir=checkpoint_dir,
            step=max(1, current_step),
            suffix="interrupt",
        )
        print(f"\nKeyboardInterrupt. Saved checkpoint to {ckpt_path}")
        return model

    return model


def save_checkpoint(model, checkpoint_dir, step, suffix=""):
    checkpoint_dir = Path(checkpoint_dir)
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    suffix = f"-{suffix}" if suffix else ""
    ckpt_path = (
        checkpoint_dir /
        f"qwen3-0.6B-rlvr-grpo-step{step:05d}{suffix}.pth"
    )
    torch.save(model.state_dict(), ckpt_path)
    return ckpt_path


def append_csv_metrics(
    csv_log_path,
    step_idx,
    total_steps,
    loss,
    reward_avg,
    avg_response_len,
):
    if not csv_log_path.exists():
        csv_log_path.write_text(
            "step,total_steps,loss,reward_avg,avg_response_len\n",
            encoding="utf-8",
        )
    with csv_log_path.open("a", encoding="utf-8") as f:
        f.write(
            f"{step_idx},{total_steps},{loss:.6f},{reward_avg:.6f},"
            f"{avg_response_len:.6f}\n"
        )

In [ ]:
device = get_device()
model.to(device)

torch.manual_seed(0)

train_rlvr_grpo(
    model=model,
    tokenizer=tokenizer,
    math_data=math_train,
    device=device,
    steps=50,
    num_rollouts=4,
    max_new_tokens=512,
    temperature=0.8,
    top_p=0.9,
    lr=1e-5,
    checkpoint_every=5,
    checkpoint_dir=".",
    csv_log_path="train_rlvr_grpo_metrics.csv",
)

Using NVIDIA CUDA GPU
[Step 1/50] loss=-0.0000 reward_avg=0.000 avg_resp_len=5.2
[Step 2/50] loss=-0.0000 reward_avg=0.000 avg_resp_len=5.8
[Step 3/50] loss=3.5188 reward_avg=0.500 avg_resp_len=23.0
[Step 4/50] loss=-0.0000 reward_avg=0.000 avg_resp_len=1.5
[Step 5/50] loss=14.2968 reward_avg=0.500 avg_resp_len=233.2
Saved checkpoint to qwen3-0.6B-rlvr-grpo-step00005.pth
[Step 6/50] loss=1.5443 reward_avg=0.250 avg_resp_len=355.0
[Step 7/50] loss=-0.0000 reward_avg=0.000 avg_resp_len=494.8
[Step 8/50] loss=3.2774 reward_avg=0.750 avg_resp_len=175.8
[Step 9/50] loss=-0.0000 reward_avg=0.000 avg_resp_len=248.8
[Step 10/50] loss=-0.0000 reward_avg=0.000 avg_resp_len=469.8
[Step 10] sample outputs
  1) reward=0.000 len=512:  To determine how much Tim should invest to reach \$60,000 in 5 years with quarterly compounding at an annual interest rate of 7%, we can use the formula for compound interest:\n\n\[\nA = P \left(1 + \frac{r}{n}\right)^{nt}\n\]\n\nWhere:\n- \( A \) is the amount of mone

In [1]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedModel

In [6]:
# # MODEL_NAME = "Qwen/Qwen2.5-3B"
# MODEL_NAME = "Qwen/qwen3-0.6B-base"

In [7]:
# policy_model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,
#     device_map=0,
# )
# # reference_model = AutoModelForCausalLM.from_pretrained(
# #     MODEL_NAME,
# #     attn_implementation="flash_attention_2",
# #     torch_dtype=torch.bfloat16,
# #     device_map=0,
# # )

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

- Hmm yeah this is like a non-trivial amount of tooling with either repo
- Raschka's seems friendlier, but more complex than I would want in some areas for sure.
- Can I achieve my hacking goals here using Raschka's code?
- Seems like it comes down to if I can make sense of the policy gradient part of his code and modify it. 